In [1]:
import pandas as pd
import loader as loader
import cleaner as cleaner

%load_ext autoreload
%autoreload 2

In [2]:
# =============== LOADER =============== #
load = False
try: 
    merged_df = pd.read_pickle('./data/Processed/merged_movie_bechdel.pkl').drop(columns=['index'])
    load = True
except: 
    # Load data 
    character_df, movie_df = loader.load_CMU_dataset()
    bechdel_df = loader.load_bechdel_dataset()

In [3]:
# =============== CLEANER =============== #
if not load:
    # Clean datasets
    movie_df = cleaner.clean_movie_df(movie_df)
    bechdel_df = cleaner.clean_bechdel_df(bechdel_df)
    
    # Merge data 
    merged_df = pd.merge(movie_df, bechdel_df, on=['movie_title', 'year'], how='left')
    merged_df = merged_df.dropna(subset=['bechdel_rating'])
    merged_df = merged_df.reset_index()
    
    # Save merged data 
    merged_df.to_pickle('./data/Processed/merged_movie_bechdel.pkl')

In [4]:
display(merged_df)

# =============== ANALYSIS =============== #
# TODO: Add analysis here

,wikiID,freebaseID,movie_title,movie_release_date,movie_bo_revenue,movie_runtime,fbid_languages,fbid_countries,fbid_genres,year,imdbid,bechdel_rating
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001.0,0228333,3.0
1,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa...",1989.0,0097499,3.0
2,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""...",1964.0,0058331,3.0
3,12053509,/m/02vn81r,Loverboy,1989-04-28,3960327.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/02l7c8"": ""...",1989.0,0097790,3.0
4,1369204,/m/04x8zs,Juarez,1939-01-01,NaN,125.0,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/09c7w0"": ""United States of America""}","{""/m/04xvh5"": ""Costume drama"", ""/m/03bxz7"": ""B...",1939.0,0031516,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3934,303933,/m/01s7w3,Twister,1996-05-10,494471524.0,113.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03btsm8"": ""Action/Adventure"", ""/m/01drsx""...",1996.0,0117998,3.0
3935,25920477,/m/0b6lqyd,Source Code,2011-03-11,147332697.0,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2011.0,0945513,1.0
3936,1191380,/m/04f_y7,Wilde,1997-01-01,2158775.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...","{""/m/0hn10"": ""LGBT"", ""/m/017fp"": ""Biography"", ...",1997.0,0120514,3.0
3937,54540,/m/0f7hw,Coming to America,1988-06-29,288752301.0,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/03p5xs"": ""...",1988.0,0094898,3.0
